# **BUSINESS REPORT**

The analysis for this report was carried out in two parts.

1. **Financial Analysis:** which highlights the performance metrics centered around Delivr's finances.
2. **Business to Customer Analysis:** this is a more user centric analysis showing the company's performance in relation to customer behaviour.

## **Finincial Analysis**

1. <u>Total Revenue, Cost and Profit</u>

In [1]:
WITH revenue_CTE AS (
SELECT m.meal_id,
	   SUM(meal_price * order_quantity) AS revenue 
FROM meals AS m
INNER JOIN orders AS o
	ON m.meal_id = o.meal_id
GROUP BY m.meal_id),-- revenue per meal
cost_CTE AS ( -- 
SELECT m.meal_id,
	   SUM(meal_cost * stocked_quantity) AS cost
FROM meals AS m
INNER JOIN stock AS s
	ON m.meal_id = s.meal_id
GROUP BY m.meal_id) -- cost per meal
SELECT FORMAT(SUM(revenue), 'C') AS total_revenue, -- Format as currency
	   FORMAT(SUM(cost), 'C') as total_cost,
	   FORMAT(SUM(revenue) -  SUM(cost), 'C') AS total_profit 
FROM revenue_CTE AS r
INNER JOIN cost_CTE AS c
	ON r.meal_id = c.meal_id;

(1 row affected)

Total execution time: 00:00:00.145

total_revenue,total_cost,total_profit
"$260,226.75","$92,133.00","$168,093.75"


2. <u>Revenue, Cost and Profit by month</u>

In [2]:
WITH revenue_CTE AS ( 
SELECT DATENAME(month, order_date) AS month,
	   SUM(meal_price * order_quantity) AS revenue 
FROM meals AS m
INNER JOIN orders AS o
	ON m.meal_id = o.meal_id
GROUP BY DATENAME(month,o.order_date)), -- revenue per month
cost_CTE  AS ( 
SELECT DATENAME(month, stocking_date) AS month,
	   DATEPART(month, stocking_date) AS month_number,
	   SUM(meal_cost * stocked_quantity) AS cost
FROM meals AS m
INNER JOIN stock AS s
	ON m.meal_id = s.meal_id
GROUP BY DATENAME(month, stocking_date), DATEPART(month, stocking_date)) -- Cost per month
SELECT r.month,
	   FORMAT(revenue, 'C') AS revenue, -- Format as currency
	   FORMAT(cost, 'C') as cost,
	   FORMAT(revenue - cost, 'C') AS profit 
FROM revenue_CTE AS r
INNER JOIN cost_CTE AS c
	ON r.month = c.month
ORDER BY c.month_number;

(7 rows affected)

Total execution time: 00:00:00.471

month,revenue,cost,profit
June,"$6,296.25","$2,222.75","$4,073.50"
July,"$10,169.75","$3,594.25","$6,575.50"
August,"$15,340.00","$5,365.75","$9,974.25"
September,"$23,819.50","$8,480.00","$15,339.50"
October,"$35,576.25","$12,488.75","$23,087.50"
November,"$60,130.00","$21,387.00","$38,743.00"
December,"$108,895.00","$38,594.50","$70,300.50"


3.  <u>Month-Over-Month Analysis for Revenue & Profit</u>

Showing the percentage increase/decrease between the previous and current month.

In [3]:
WITH revenue_CTE AS ( 
SELECT DATENAME(month, order_date) AS month,
	   SUM(meal_price * order_quantity) AS revenue 
FROM meals AS m
INNER JOIN orders AS o
	ON m.meal_id = o.meal_id
GROUP BY DATENAME(month,o.order_date)), -- revenue per month
cost_CTE  AS (  
SELECT DATENAME(month, stocking_date) AS month,
	   DATEPART(month, stocking_date) AS month_number,
	   SUM(meal_cost * stocked_quantity) AS cost
FROM meals AS m
INNER JOIN stock AS s
	ON m.meal_id = s.meal_id
GROUP BY DATENAME(month, stocking_date), DATEPART(month, stocking_date)), -- Cost per month
MOM_CTE AS (
SELECT r.month AS month,
	   c.month_number AS month_number,
	   revenue AS Current_revenue,
	   LAG(revenue) OVER(ORDER BY c.month_number) AS previous_revenue,
	   revenue - cost AS current_profit,
	   LAG(revenue - cost) OVER(ORDER BY c.month_number) AS previous_profit
FROM revenue_CTE AS r
INNER JOIN cost_CTE AS c
	ON r.month = c.month)
SELECT month,
	   ROUND((current_revenue - previous_revenue) / (previous_revenue-1), 2) AS revenue_percent_MOM,
	   ROUND((current_profit - previous_profit) / (previous_profit-1), 2) AS profit_percent_MOM
FROM MOM_CTE
ORDER BY month_number;

(7 rows affected)

Total execution time: 00:00:00.372

month,revenue_percent_MOM,profit_percent_MOM
June,NULL,NULL
July,0.62,0.61
August,0.51,0.52
September,0.55,0.54
October,0.49,0.51
November,0.69,0.68
December,0.81,0.81


4. <u>Revenue, Cost amd Profit by Eatery</u>

Showing eateries arranged in order of profit contribution from most to least.

In [2]:
WITH revenue_CTE AS ( 
SELECT eatery,
	   SUM(meal_price * order_quantity) AS revenue 
FROM meals AS m
INNER JOIN orders AS o
	ON m.meal_id = o.meal_id
GROUP BY eatery), -- revenue per eatery
cost_CTE  AS ( 
SELECT eatery,
	   SUM(meal_cost * stocked_quantity) AS cost
FROM meals AS m
INNER JOIN stock AS s
	ON m.meal_id = s.meal_id
GROUP BY eatery)   -- Cost per eatery
SELECT REPLACE(r.eatery, '''', '') AS eatery, -- removing quotes from string
	   FORMAT(revenue, 'C') AS revenue, -- Format as currency
	   FORMAT(cost, 'C') AS Cost,
	   FORMAT(revenue - cost, 'C') AS profit,  
	   ROUND((revenue - cost) / SUM(revenue - cost) OVER(),2)  AS profit_percentage
FROM revenue_CTE AS r
INNER JOIN cost_CTE AS c
	ON r.eatery = c.eatery
ORDER BY profit_percentage DESC; 

(5 rows affected)

Total execution time: 00:00:00.033

eatery,revenue,Cost,profit,profit_percentage
Bean Me Up Scotty,"$60,652.50","$13,798.00","$46,854.50",0.28
Burgatorio,"$71,806.25","$34,271.75","$37,534.50",0.22
The Moon Wok,"$60,770.50","$24,734.00","$36,036.50",0.21
Leaning Tower of Pizza,"$47,702.00","$17,009.50","$30,692.50",0.18
Life of Pie,"$19,295.50","$2,319.75","$16,975.75",0.1


5. <u>Average revenue per user (ARPU)</u>

Showing the average amount each customer pays per month.

In [3]:
WITH revenue_CTE AS (
SELECT  DATENAME(MONTH, order_date) AS month,
        DATEPART(MONTH, order_date) AS month_number,
	    SUM(meal_price * order_quantity) AS revenue,
		COUNT(DISTINCT user_id) AS user_count
FROM meals AS m
JOIN orders AS O
ON m.meal_id = o.meal_id
GROUP BY DATENAME(MONTH, order_date), DATEPART(MONTH, order_date))
SELECT month,
	   FORMAT(ROUND(revenue / user_count, 2), 'C') AS ARPU -- Format as currency
FROM revenue_CTE
ORDER BY month_number;

(7 rows affected)

Total execution time: 00:00:00.174

month,ARPU
June,$51.19
July,$45.00
August,$45.52
September,$48.71
October,$51.63
November,$63.70
December,$85.95


## **Business to Customer Analysis**

1. <u>Number of Orders by Month</u>

In [4]:
SELECT  DATENAME(MONTH, order_date) AS month,
	    COUNT(DISTINCT order_id) AS order_count
FROM meals AS m
JOIN orders AS O
ON m.meal_id = o.meal_id
GROUP BY DATENAME(MONTH, order_date), DATEPART(MONTH, order_date)
ORDER BY DATEPART(MONTH, order_date);

(7 rows affected)

Total execution time: 00:00:00.081

month,order_count
June,282
July,445
August,670
September,1005
October,1549
November,2584
December,4816


2. <u>Top 10 Most Popular Meals</u>

In [5]:
SELECT TOP 10
	   REPLACE(eatery, '''', '') AS eatery, -- removing quotes from string
	   m.meal_id,
	   COUNT(DISTINCT order_id) AS order_count
FROM meals AS m
INNER JOIN orders AS o
	ON m.meal_id = o.meal_id
GROUP BY eatery, m.meal_id
ORDER BY order_count DESC;

(10 rows affected)

Total execution time: 00:00:00.041

eatery,meal_id,order_count
Life of Pie,17,1566
Burgatorio,4,1562
Life of Pie,18,1541
Bean Me Up Scotty,9,1535
Bean Me Up Scotty,8,1534
Bean Me Up Scotty,11,1529
Leaning Tower of Pizza,3,1526
Leaning Tower of Pizza,1,1518
Burgatorio,5,1518
Bean Me Up Scotty,10,1514


3. <u>Number of  New Customers per Month</u>

In [1]:
SELECT DATENAME(MONTH, reg_date) AS month,
	   COUNT(DISTINCT user_id) AS new_users
FROM (
	SELECT user_id,
		   MIN(order_date) AS reg_date
	FROM meals AS m
	INNER JOIN orders AS o
		ON m.meal_id = o.meal_id
	GROUP BY user_id) AS subquery
GROUP BY DATENAME(MONTH, reg_date), DATEPART(MONTH, reg_date)
ORDER BY DATEPART(MONTH, reg_date);

(7 rows affected)

Total execution time: 00:00:00.168

month,new_users
June,123
July,140
August,157
September,176
October,199
November,231
December,278


4. <u>User Activity by Month</u>

In [7]:
SELECT months_active, 
	   COUNT(user_id) AS user_count
FROM(
	SELECT user_id,
		   COUNT(DISTINCT (MONTH(order_date))) AS months_active
FROM orders
GROUP BY user_id) AS subqery
GROUP BY months_active
ORDER BY months_active;

(7 rows affected)

Total execution time: 00:00:00.044

months_active,user_count
1,292
2,242
3,247
4,232
5,162
6,83
7,46


5. <u>Customer Growth Rate by Month</u>

Increase in the number of customers from the previous month.

In [8]:
WITH user_activity AS (
SELECT order_date,
       COUNT(user_id) AS user_count
FROM meals AS m
INNER JOIN orders AS o
	ON m.meal_id = o.meal_id
GROUP by order_date),
MAU AS ( 
SELECT DATENAME(MONTH, order_date) AS month,
	   SUM(user_count) AS Active_users,
	   LAG(SUM(user_count)) OVER(ORDER BY DATEPART(MONTH, order_date)) AS previous_month_users
FROM user_activity
GROUP BY DATENAME(MONTH, order_date), DATEPART(MONTH, order_date))
SELECT month,
	   Active_users,
	   previous_month_users,
	   ROUND(CAST((Active_users - previous_month_users) AS float) / previous_month_users, 2) AS growth_rate
FROM MAU;

(7 rows affected)

Total execution time: 00:00:00.037

month,Active_users,previous_month_users,growth_rate
June,703,NULL,NULL
July,1107,703,0.57
August,1680,1107,0.52
September,2609,1680,0.55
October,3929,2609,0.51
November,6579,3929,0.67
December,12065,6579,0.83


6. <u>Customer Retention Rate</u>

The number of active users who remained active in the next month.

In [9]:
WITH user_activity AS (
SELECT DISTINCT CONVERT(DATE, (DATEADD(MONTH, DATEDIFF(MONTH, 0, order_date) , 0))) AS month, -- Truncate date to first day of the month
	   user_id
FROM orders)
SELECT DATENAME(MONTH, prev.month) AS month,
	   COUNT(DISTINCT prev.user_id) AS users, 
	   COUNT(DISTINCT curr.user_id) AS retained_users, -- Users retained from previous month
	   ROUND (CAST(COUNT(DISTINCT curr.user_id) AS FLOAT) /
	   COUNT(DISTINCT prev.user_id), 2) AS retention_rate
FROM user_activity AS prev
LEFT JOIN user_activity AS curr
	ON prev.user_id = curr.user_id
	AND prev.month = (DATEADD(MONTH, -1, curr.month)) -- Looking one month ahead
GROUP BY prev.month
ORDER BY prev.month;

Warning: Null value is eliminated by an aggregate or other SET operation.

(7 rows affected)

Total execution time: 00:00:00.074

month,users,retained_users,retention_rate
June,123,86,0.7
July,226,158,0.7
August,337,257,0.76
September,489,404,0.83
October,689,621,0.9
November,944,909,0.96
December,1267,0,0
